# **Comparison of Different Data Reading Methods:**

In [ ]:
import os
import time

In [ ]:
os.path.getsize('Rate.csv')

17825792

In [ ]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('Rate.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.030173540115356445 sec


In [ ]:
import pandas as pd
start = time.time()
df = pd.read_csv('Rate.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  0.5553486347198486 sec


In [ ]:
!pip install modin[ray]
!pip install modin[pandas]
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('Rate.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.8 MB/s 
     |████████████████████████████████| 956 kB 44.3 MB/s 
     |████████████████████████████████| 956 kB 18.0 MB/s 
     |████████████████████████████████| 761 kB 52.0 MB/s 
     |████████████████████████████████| 60.5 MB 1.2 MB/s 
     |████████████████████████████████| 4.1 MB 50.8 MB/s 
     |████████████████████████████████| 8.8 MB 43.8 MB/s 
     |████████████████████████████████| 201 kB 45.0 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 128 kB 43.6 MB/s 
     |████████████████████████████████| 90 kB 8.7 MB/s 
     |████████████████████████████████| 3.0 MB 42.8 MB/s 
     |████████████████████████████████| 291 kB 56.5 MB/s 
     |████████████████████████████████| 58 kB 4.2 MB/s 
     |████████████████████████████████| 468 kB 49.7 MB/s 
  Created wheel for gpustat: filename=

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2022-10-25 18:12:52,761	INFO worker.py:1515 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2022-10-25 18:12:54,715	WARNING __init__.py:193 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3

Read csv with modin and ray:  1.8111183643341064 sec


Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


**Conclusion:** Dask has the slowest computational time among Pandas, Modin and Ray which is 0.030 seconds.




---



# **Basic Data Cleaning :**

In [1]:
from dask import dataframe as dd
df = dd.read_csv('Rate.csv',delimiter=',')
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 24 entries, BusinessYear to RowNumber
dtypes: object(10), float64(9), int64(5)

In [9]:
#No. of Rows
len(df)

313110

In [ ]:
#No. of Columns
len(df.columns)

24

In [10]:
#to remove special character
df.columns=df.columns.str.replace('[#,@,&]','')
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [11]:
data=df.columns
data

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'FederalTIN', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'RowNumber'],
      dtype='object')

# **Basic Data Validation :**

In [12]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [13]:
%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing utility.py


In [14]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: Rate
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - BusinessYear
      - StateCode
      - IssuerId
      - SourceName
      - VersionNum
      - ImportDate
      - IssuerId2
      - FederalTIN
      - RateEffectiveDate
      - RateExpirationDate
      - PlanId
      - RatingAreaId
      - Tobacco
      - Age
      - IndividualRate
      - IndividualTobaccoRate
      - Couple
      - PrimarySubscriberAndOneDependent
      - PrimarySubscriberAndTwoDependents
      - PrimarySubscriberAndThreeOrMoreDependents
      - CoupleAndOneDependent
      - CoupleAndTwoDependents
      - CoupleAndThreeOrMoreDependents
      - RowNumber

Writing store.yaml


In [20]:
!pip install pyyaml
# Reading config file
import utility as util
import yaml
with open("store.yaml", "r") as stream:
    try:
        print(yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
{'file_type': 'csv', 'dataset_name': 'file', 'file_name': 'Rate', 'table_name': 'edsurv', 'inbound_delimiter': ',', 'outbound_delimiter': '|', 'skip_leading_rows': 1, 'columns': ['BusinessYear - StateCode - IssuerId - SourceName - VersionNum - ImportDate - IssuerId2 - FederalTIN - RateEffectiveDate - RateExpirationDate - PlanId - RatingAreaId - Tobacco - Age - IndividualRate - IndividualTobaccoRate - Couple - PrimarySubscriberAndOneDependent - PrimarySubscriberAndTwoDependents - PrimarySubscriberAndThreeOrMoreDependents - CoupleAndOneDependent - CoupleAndTwoDependents - CoupleAndThreeOrMoreDependents - RowNumber']}


In [25]:
# Reading process of the file using Dask we got the following columns
data

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'FederalTIN', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'RowNumber'],
      dtype='object')

Column validation successful.

# **Writing the file in pipe**

In [31]:
import datetime
import csv
import gzip
from dask import dataframe as dd
df = dd.read_csv('Rate.csv',delimiter=',',dtype={'IssuerId2': 'float64','RowNumber': 'float64'})
# Write csv in gz format in pipe separated text file (|)
df.to_csv('Rate.csv.gz',sep='|',header=True,index=False, quoting=csv.QUOTE_ALL, compression='gzip',quotechar='"',doublequote=True,line_terminator='\n')

['/content/Rate.csv.gz/0.part',
 '/content/Rate.csv.gz/1.part',
 '/content/Rate.csv.gz/2.part',
 '/content/Rate.csv.gz/3.part',
 '/content/Rate.csv.gz/4.part']

In [32]:
#number of files in gz format folder
import os
entries = os.listdir('Rate.csv.gz/')
for entry in entries:
    print(entry)

3.part
2.part
4.part
1.part
0.part


In [33]:
#size of the gz format folder
os.path.getsize('Rate.csv.gz')

4096